### Удаление выбросов


In [2]:
from google.colab import files
uploaded = files.upload()

Saving test_df.csv to test_df.csv
Saving train_df.csv to train_df.csv


In [3]:
import pandas as pd

In [4]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.4 MB/s eta 0:00:00


In [5]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.linear_model import SGDClassifier
#from sklearn import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from catboost import cv
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [6]:
from sklearn.tree import DecisionTreeClassifier

In [7]:
train = pd.read_csv('train_df.csv')
train.head(1)

,difference_max,difference_mean,is_alcohol_max,is_own_trademark_max,purchase_sum_max,purchase_sum_mean,purchase_sum_min,regular_points_received_mean,regular_points_spent_mean,age,target,gender_M,gender_U,year_first_issue,month_first_issue,weekday_first_issue,year_first_redeem,month_first_redeem,weekday_first_redeem
0,82.0,25.75,0,1,1007.0,700.75,419.0,98.175,0.0,45,0,0,1,2017,8,5,2018,1,3


In [8]:
test = pd.read_csv('test_df.csv')
test.head(1)

,difference_max,difference_mean,is_alcohol_max,is_own_trademark_max,purchase_sum_max,purchase_sum_mean,purchase_sum_min,regular_points_received_mean,regular_points_spent_mean,age,gender_M,gender_U,year_first_issue,month_first_issue,weekday_first_issue,year_first_redeem,month_first_redeem,weekday_first_redeem
0,33.0,5.166667,1,1,581.0,326.833333,132.0,9.055556,-4.722222,83,0,1,2018,7,1,2018,9,4


In [9]:
train.shape

(140027, 19)

In [10]:
test.shape

(60012, 18)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_data, test_data = train_test_split(train, test_size = 0.2, random_state = 123)
train_data, valid_data = train_test_split(train_data, test_size = 0.2, random_state = 123)

In [13]:
print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

(89616, 19)
(22405, 19)
(28006, 19)


In [14]:
test_data.head()

,difference_max,difference_mean,is_alcohol_max,is_own_trademark_max,purchase_sum_max,purchase_sum_mean,purchase_sum_min,regular_points_received_mean,regular_points_spent_mean,age,target,gender_M,gender_U,year_first_issue,month_first_issue,weekday_first_issue,year_first_redeem,month_first_redeem,weekday_first_redeem
78102,8.0,3.000000,0,1,526.00,236.032258,23.00,6.967742,0.000000,79,1,0,1,2017,11,1,2018,6,3
137586,48.0,20.333333,1,1,2497.00,1696.000000,1173.00,198.900000,0.000000,36,0,0,1,2018,4,1,2019,8,5
118419,18.0,3.166667,0,1,744.00,259.246333,68.99,7.670000,-3.266667,21,0,0,1,2018,5,6,2018,7,5
122612,47.0,18.800000,0,1,1086.00,778.200000,477.00,77.580000,0.000000,65,1,0,0,2018,10,3,2018,11,2
20207,22.0,5.277778,0,1,1232.64,597.895000,46.40,76.227778,0.000000,52,1,0,1,2018,12,2,2019,9,4


In [15]:
target_train = train_data['target']
target_test = test_data['target']
target_valid = valid_data['target']



In [16]:
features_train = train_data.drop(['target'], axis = 1)
features_test = test_data.drop(['target'], axis = 1)
features_valid = valid_data.drop(['target'], axis = 1)

In [17]:
models = []
columns = ['model', 'roc auc (train)', 'roc auc(test)', 'accuracy(train)', 'accuracy(test)']
models = pd.DataFrame(data = models, columns = columns)

In [18]:
all_models = [SGDClassifier(), DecisionTreeClassifier(), RandomForestClassifier(), XGBClassifier(), LGBMClassifier(),CatBoostClassifier()]
all_df = ['train']#

In [19]:
def models_result(model): # 
    global features_train, features_test, target_train, target_test
    #global features_train_auto, features_test_auto, target_train_auto, target_test_auto
    name_model = str(model)[:-2]
    
    model.fit(features_train, target_train)
    result_train= model.predict(features_train)
    result_test= model.predict(features_test)
    roc_auc_train = roc_auc_score(target_train, result_train)
    accuracy_train = accuracy_score(target_train, result_train)
    roc_auc_test = roc_auc_score(target_test, result_test)
    accuracy_test = accuracy_score(target_test, result_test)

    tmp = [name_model, roc_auc_train, accuracy_train, roc_auc_test, accuracy_test]
    models.loc[len(models)]= tmp
    return models

In [20]:
for i in all_models:
    models_result(i)
models

Learning rate set to 0.070243
0:	learn: 0.6926281	total: 72.4ms	remaining: 1m 12s
1:	learn: 0.6923168	total: 95.3ms	remaining: 47.6s
2:	learn: 0.6919795	total: 118ms	remaining: 39.1s
3:	learn: 0.6917039	total: 141ms	remaining: 35s
4:	learn: 0.6915381	total: 163ms	remaining: 32.4s
5:	learn: 0.6914280	total: 184ms	remaining: 30.5s
6:	learn: 0.6912945	total: 234ms	remaining: 33.2s
7:	learn: 0.6911574	total: 258ms	remaining: 32s
8:	learn: 0.6910361	total: 282ms	remaining: 31s
9:	learn: 0.6909334	total: 304ms	remaining: 30.1s
10:	learn: 0.6908538	total: 327ms	remaining: 29.4s
11:	learn: 0.6907761	total: 351ms	remaining: 28.9s
12:	learn: 0.6907065	total: 374ms	remaining: 28.4s
13:	learn: 0.6904786	total: 396ms	remaining: 27.9s
14:	learn: 0.6903871	total: 428ms	remaining: 28.1s
15:	learn: 0.6903073	total: 453ms	remaining: 27.9s
16:	learn: 0.6900653	total: 475ms	remaining: 27.5s
17:	learn: 0.6899924	total: 497ms	remaining: 27.1s
18:	learn: 0.6899281	total: 521ms	remaining: 26.9s
19:	learn: 0.6

,model,roc auc (train),roc auc(test),accuracy(train),accuracy(test)
0,SGDClassifier,0.500153,0.482034,0.499940,0.487146
1,DecisionTreeClassifier,1.000000,1.000000,0.504814,0.505177
2,RandomForestClassifier,1.000000,1.000000,0.505525,0.506963
3,XGBClassifier,0.530347,0.540852,0.508749,0.516104
4,LGBMClassifier,0.602120,0.609355,0.508219,0.513461
5,<catboost.core.CatBoostClassifier object at 0x...,0.662715,0.666678,0.504836,0.507927


#### LocalOutlierFactor

In [21]:
import numpy as np
from sklearn.neighbors import LocalOutlierFactor

In [23]:
lof = LocalOutlierFactor(n_neighbors=20, contamination=.03)

In [24]:
y_pred = lof.fit_predict(features_train)

In [25]:
lofs_index=where(y_pred==-1)

In [26]:
features_train

,difference_max,difference_mean,is_alcohol_max,is_own_trademark_max,purchase_sum_max,purchase_sum_mean,purchase_sum_min,regular_points_received_mean,regular_points_spent_mean,age,gender_M,gender_U,year_first_issue,month_first_issue,weekday_first_issue,year_first_redeem,month_first_redeem,weekday_first_redeem
4416,31.0,7.846154,0,1,1201.00,489.206154,160.00,38.838462,-38.461538,62,0,1,2017,10,4,2018,1,4
76452,34.0,22.000000,0,0,269.95,148.275000,64.18,2.225000,-1.750000,61,0,1,2017,7,0,2017,8,6
113152,19.0,4.041667,0,1,939.00,200.833333,57.00,9.354167,-3.375000,62,0,1,2018,8,3,2018,10,5
47101,26.0,2.866667,1,1,901.00,250.500000,23.00,8.793333,-8.866667,31,0,1,2017,6,4,2018,2,3
80552,18.0,5.500000,0,1,3995.00,774.699375,144.97,113.543750,-59.750000,43,0,0,2018,6,1,2018,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9318,44.0,9.181818,0,1,575.00,157.636364,31.00,4.809091,-4.818182,47,0,1,2018,4,0,2019,2,2
101764,4.0,0.961538,1,1,868.98,182.834231,19.00,5.519231,-6.346154,42,0,1,2019,2,6,2019,3,2
71724,5.0,2.500000,1,1,2578.36,1510.295000,442.23,140.250000,-209.000000,30,0,0,2017,4,4,2017,7,3
113564,13.0,2.545455,0,1,844.08,409.951818,179.99,25.836364,-16.363636,52,0,1,2018,9,6,2019,3,3


In [27]:
train_data['outliers'] = y_pred
train_data.outliers.value_counts()

 1    86927
-1     2689
Name: outliers, dtype: int64

In [28]:
train_data_lof = train_data.loc[train_data.outliers == 1]

In [29]:
train_data = train_data.drop(['outliers'], axis = 1)

#### IsolationForest

In [30]:
from sklearn.ensemble import IsolationForest

In [31]:

clf = IsolationForest( max_samples=100, random_state = 1, contamination= 'auto')
preds = clf.fit_predict(train_data)
train_data['outliers'] = preds
train_data.outliers.value_counts()

 1    59037
-1    30579
Name: outliers, dtype: int64

In [32]:
train_data_if = train_data.loc[train_data.outliers == 1]
train_data = train_data.drop(['outliers'], axis = 1)

In [37]:
from sklearn.cluster import DBSCAN

In [38]:

outlier_detection = DBSCAN(min_samples=3, eps=1)
clusters = outlier_detection.fit_predict(train_data)

clusters

array([-1, -1, -1, ..., -1, -1, -1])

In [39]:
train_data['outliers'] = clusters
train_data.outliers.value_counts()

-1    89616
Name: outliers, dtype: int64

In [43]:
target_train = train_data['target']
features_train = train_data.drop(['target'], axis = 1)

target_train_lof = train_data_lof['target']
features_train_lof = train_data_lof.drop(['target'], axis = 1)

target_train_if = train_data_if['target']
features_train_if = train_data_if.drop(['target'], axis = 1)

In [57]:
models = []
columns = ['data', 'roc auc (train)', 'roc auc(test)']
models = pd.DataFrame(data = models, columns = columns)

In [45]:
features_train = features_train.drop(['outliers'], axis = 1)

In [53]:
features_train_lof = features_train_lof.drop(['outliers'], axis = 1)
features_train_if = features_train_if.drop(['outliers'], axis = 1)

In [52]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89616 entries, 4416 to 10041
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   difference_max                89616 non-null  float64
 1   difference_mean               89616 non-null  float64
 2   is_alcohol_max                89616 non-null  int64  
 3   is_own_trademark_max          89616 non-null  int64  
 4   purchase_sum_max              89616 non-null  float64
 5   purchase_sum_mean             89616 non-null  float64
 6   purchase_sum_min              89616 non-null  float64
 7   regular_points_received_mean  89616 non-null  float64
 8   regular_points_spent_mean     89616 non-null  float64
 9   age                           89616 non-null  int64  
 10  gender_M                      89616 non-null  int64  
 11  gender_U                      89616 non-null  int64  
 12  year_first_issue              89616 non-null  int64  
 13

In [60]:
def models_result(model): # 
    global features_train, features_test, target_train, target_test, features_train_lof, target_train_lof, features_train_if, target_train_if
    #global features_train_auto, features_test_auto, target_train_auto, target_test_auto

    models = []
    columns = ['data', 'roc auc (train)', 'roc auc(test)']
    models = pd.DataFrame(data = models, columns = columns)

    model.fit(features_train, target_train)
    result_train= model.predict(features_train)
    result_test= model.predict(features_test)
    roc_auc_train = roc_auc_score(target_train, result_train)
    roc_auc_test = roc_auc_score(target_test, result_test)
    tmp = ['all data', roc_auc_train, roc_auc_test]
    models.loc[len(models)]= tmp

    model.fit(features_train_lof, target_train_lof)
    result_train= model.predict(features_train_lof)
    result_test= model.predict(features_test)
    roc_auc_train = roc_auc_score(target_train_lof, result_train)
    roc_auc_test = roc_auc_score(target_test, result_test)
    tmp = ['LOF', roc_auc_train, roc_auc_test]
    models.loc[len(models)]= tmp

    model.fit(features_train_if, target_train_if)
    result_train= model.predict(features_train_if)
    result_test= model.predict(features_test)
    roc_auc_train = roc_auc_score(target_train_if, result_train)
    roc_auc_test = roc_auc_score(target_test, result_test)
    tmp = ['Isolation Forest', roc_auc_train, roc_auc_test]
    models.loc[len(models)]= tmp

    return models

In [61]:
models_result(DecisionTreeClassifier())

,data,roc auc (train),roc auc(test)
0,all data,1.0,0.503657
1,LOF,1.0,0.503280
2,Isolation Forest,1.0,0.502716


In [62]:
models_result(RandomForestClassifier())

,data,roc auc (train),roc auc(test)
0,all data,1.0,0.503729
1,LOF,1.0,0.509162
2,Isolation Forest,1.0,0.497785


In [63]:
models_result(CatBoostClassifier())

Learning rate set to 0.070243
0:	learn: 0.6926281	total: 26.9ms	remaining: 26.9s
1:	learn: 0.6923168	total: 48ms	remaining: 24s
2:	learn: 0.6919795	total: 70.4ms	remaining: 23.4s
3:	learn: 0.6917039	total: 90.9ms	remaining: 22.6s
4:	learn: 0.6915381	total: 115ms	remaining: 22.9s
5:	learn: 0.6914280	total: 137ms	remaining: 22.6s
6:	learn: 0.6912945	total: 163ms	remaining: 23.1s
7:	learn: 0.6911574	total: 187ms	remaining: 23.2s
8:	learn: 0.6910361	total: 217ms	remaining: 23.9s
9:	learn: 0.6909334	total: 240ms	remaining: 23.7s
10:	learn: 0.6908538	total: 261ms	remaining: 23.5s
11:	learn: 0.6907761	total: 283ms	remaining: 23.3s
12:	learn: 0.6907065	total: 306ms	remaining: 23.3s
13:	learn: 0.6904786	total: 328ms	remaining: 23.1s
14:	learn: 0.6903871	total: 351ms	remaining: 23s
15:	learn: 0.6903073	total: 373ms	remaining: 22.9s
16:	learn: 0.6900653	total: 396ms	remaining: 22.9s
17:	learn: 0.6899924	total: 422ms	remaining: 23s
18:	learn: 0.6899281	total: 445ms	remaining: 23s
19:	learn: 0.6898

,data,roc auc (train),roc auc(test)
0,all data,0.662715,0.504836
1,LOF,0.664734,0.507991
2,Isolation Forest,0.696744,0.503095


С помощью алгоритма LOF удалось достичь более высоких результатов построенных моделей. Финальная модель будет построена на данных, обработанных 